In [150]:
from py2neo import Graph, Subgraph 
from py2neo import Node,Relationship, Path 
import pandas as pd
import re 


In [151]:
# 连接数据库
# graph = Graph('http://localhost:7474', username='neo4j', password='123456') # 旧版本
graph = Graph('bolt://localhost:7687', auth=('neo4j', '2000212YYyLLl.@'))

# 删除所有已有节点
graph.delete_all()

In [152]:
# 读取数据
df = pd.read_csv('data_with_embedding.xlsx', usecols=[ 1,2,3,4], skiprows=1, nrows=276, header=None, names=['name', 'standard_code','description','embedding'])

print("读取的CSV数据:")
print(df)

print(df.iloc[0, 3])

读取的CSV数据:
       name standard_code                                        description  \
0     预防保健科             1  预防保健科主要对管辖区域的儿童保健、疾病预防、传染病管理、健康宣教，还有监督卫生监测等进行日...   
1     全科医疗科             2  全科医学科是一个面向社区与家庭，整合临床医学、预防医学、康复医学以及人文社会学科相关内容为一...   
2        内科             3         包括呼吸内科，消化内科，心血管内科，神经内科，肿瘤科，内分泌科，血液内科，传染病科等   
3    呼吸内科专业          3.01  呼吸内科专业是治疗呼吸系统疾病的一种科室，主要涉及肺结核、支气管哮喘、上呼吸道感染、肺气肿等...   
4    消化内科专业          3.02  消化内科科是一门专门研究人体消化系统及消化疾病的临床医学专业，涵盖了消化系统疾病的预防、诊断...   
..      ...           ...                                                ...   
271     彝医学         51.04  彝族人民长期同疾病作斗争的经验总结和智慧结晶。彝族有本民族文字，现在所能收集到的最早的彝族医...   
272     傣医学         51.05  傣医学主要研究傣药学、傣医诊疗法、傣医临床医疗技术等方面的基本知识和技能，运用傣医的理法方药...   
273      其他         51.06                                          民族医学科其他科室   
274  中西医结合科            52  西医结合科综合了现代医学与传统民族医学（包括中医药学、藏医、蒙医、维医等）的精髓，通过整合中...   
275  其他业务科室            69                                             其他业务科室   

                             

In [153]:
#! pip install langchain_core.prompts
#! pip install langchain_openai

In [154]:
# from langchain_core.prompts import PromptTemplate
# from langchain_openai import ChatOpenAI
# from langchain_openai import OpenAIEmbeddings

# import os
# os.environ["http_proxy"] = "http://localhost:7890"
# os.environ["https_proxy"] = "http://localhost:7890"

# api_key = os.environ["OPENAI_API_KEY"]
# print(api_key)

# llm = OpenAIEmbeddings(model="text-embedding-3-large",api_key=api_key)


In [155]:
def categorize_by_code(code):
    if re.match(r'^\d{1,2}$', code):
        return '综合科室', 1
    elif re.match(r'^\d{1,2}\.\d{1,2}$', code):
        return '专业科室', 2
    elif re.match(r'^\d{1,2}\.\d{1,2}\.\d{1,2}$', code):
        return '相关知识', 3
    else:
        return '相关知识', 4

In [156]:
# 创建节点并存储在字典中
nodes = {}
for _, row in df.iterrows():
    name = row['name']
    standard_code = row['standard_code']
    description = row['description']
    embedding = row['embedding']
    category, level = categorize_by_code(standard_code)
    
    # 在节点属性中包含标准编码和等级
    node_properties = {'name': name, 'standard_code': standard_code, 'level': level, 'description':description, 'embedding':embedding }
    node = Node(category, **node_properties)
    
    graph.create(node)
    nodes[standard_code] = node

In [157]:
# 构建关系
relationships_data = []

# 构建关系：根据标准编码的层级关系确定包含关系
for start_code, start_node in nodes.items():
    for end_code, end_node in nodes.items():
        if start_code != end_code:
            start_parts = start_code.lstrip('0').split('.')
            end_parts = end_code.lstrip('0').split('.')
            
            # end_code 是 start_code 的子节点，并且严格下一级
            if len(end_parts) == len(start_parts) + 1 and end_parts[:-1] == start_parts:
                if start_node['level'] == 1 and end_node['level'] == 2:
                    relationships_data.append((start_node, 'contain', end_node))
                elif start_node['level'] == 2 and end_node['level'] == 3:
                    relationships_data.append((start_node, 'contain', end_node))
                elif start_node['level'] == 3 and end_node['level'] == 4:
                    relationships_data.append((start_node, 'contain', end_node))

# 创建关系
for start_node, rel_type, end_node in relationships_data:
    rel = Relationship(start_node, rel_type, end_node)
    graph.create(rel)

print("节点和关系已成功创建。")

节点和关系已成功创建。
